This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.1_tutorials/wallaroo-model-operations-tutorials/deploy/by-framework/keras).

## Wallaroo Model Upload via the Wallaroo SDK: TensorFlow keras Sequential Single IO

The following tutorial demonstrates how to upload a TensorFlow keras Sequential Single IO model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a TensorFlow keras Sequential Single IO to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* Wallaroo Version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

import datetime

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

In [3]:
workspace_name = f'keras-sequential-single-io'
pipeline_name = f'keras-sequential-single-io'

model_name = 'keras-sequential-single-io'
model_file_name = 'models/model-auto-conversion_keras_single_io_keras_sequential_model.h5'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for TensorFlow keras models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a TensorFlow Keras model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, TensorFlow keras model Required*) | Set as the `Framework.KERAS`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, TensorFlow Keras model Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, TensorFlow Keras model Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Upload Method Optional, TensorFlow model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [5]:
input_schema = pa.schema([
    pa.field('input', pa.list_(pa.float64(), list_size=10))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float64(), list_size=32))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.KERAS`.

In [6]:
framework=Framework.KERAS

model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=framework, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.......
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime........successful

Ready


Name,keras-sequential-single-io
Version,dd9ea9c5-f8a9-4ed6-bd20-23c5a744c86d
File Name,model-auto-conversion_keras_single_io_keras_sequential_model.h5
SHA,f7e49538e38bebe066ce8df97bac8be239ae8c7d2733e500c8cd633706ae95a8
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.2.0-main-5421
Architecture,x86
Acceleration,none
Updated At,2024-22-Jul 20:59:18
Workspace id,15
Workspace name,keras-sequential-single-io


In [7]:
model.config().runtime()

'flight'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [9]:
# clear the pipeline if used in a previous tutorial
pipeline.undeploy()
pipeline.clear()
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

Waiting for deployment - this will take up to 45s ............................ ok


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.1.28',
   'name': 'engine-5cf46868bd-j754n',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'keras-sequential-single-io',
      'status': 'Running',
      'version': 'a2523397-0a9f-43c8-b4c2-da4a9493e6f7'}]},
   'model_statuses': {'models': [{'name': 'keras-sequential-single-io',
      'sha': 'f7e49538e38bebe066ce8df97bac8be239ae8c7d2733e500c8cd633706ae95a8',
      'status': 'Running',
      'version': 'dd9ea9c5-f8a9-4ed6-bd20-23c5a744c86d'}]}}],
 'engine_lbs': [{'ip': '10.28.1.27',
   'name': 'engine-lb-6b59985857-jvxzw',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.28.1.29',
   'name': 'engine-sidekick-keras-sequential-single-io-15-7d9db84465-4wxwf',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [10]:
input_data = np.random.rand(10, 10)
mock_dataframe = pd.DataFrame({
    "input": input_data.tolist()
})
mock_dataframe

,input
0,"[0.49673320526081977, 0.5586423614885364, 0.42..."
1,"[0.10126060560536199, 0.7037617073251677, 0.94..."
2,"[0.026495906590471185, 0.7190501964570059, 0.4..."
3,"[0.06432205620500386, 0.9834999537432707, 0.55..."
4,"[0.20412827260252786, 0.5799663571274025, 0.00..."
5,"[0.8978152460797632, 0.6898186677480959, 0.630..."
6,"[0.9104162964560322, 0.49879122361451433, 0.59..."
7,"[0.7798859682979128, 0.8083992826656565, 0.167..."
8,"[0.6489823855615262, 0.4121655701789879, 0.031..."
9,"[0.2535415531735996, 0.9149270943671959, 0.785..."


In [11]:
pipeline.infer(mock_dataframe)

,time,in.input,out.output,anomaly.count
0,2024-07-22 20:59:50.780,"[0.4967332053, 0.5586423615, 0.4240760061, 0.9...","[0.021340997889637947, 0.035429518669843674, 0...",0
1,2024-07-22 20:59:50.780,"[0.1012606056, 0.7037617073, 0.9431607941, 0.1...","[0.03815283998847008, 0.019792448729276657, 0....",0
2,2024-07-22 20:59:50.780,"[0.0264959066, 0.7190501965, 0.4100749177, 0.3...","[0.02965729497373104, 0.027410384267568588, 0....",0
3,2024-07-22 20:59:50.780,"[0.0643220562, 0.9834999537, 0.5518424679, 0.8...","[0.029501374810934067, 0.027281081303954124, 0...",0
4,2024-07-22 20:59:50.780,"[0.2041282726, 0.5799663571, 0.0084905288, 0.9...","[0.023593084886670113, 0.036172326654195786, 0...",0
5,2024-07-22 20:59:50.780,"[0.8978152461, 0.6898186677, 0.6309531026, 0.0...","[0.027886733412742615, 0.02047501690685749, 0....",0
6,2024-07-22 20:59:50.780,"[0.9104162965, 0.4987912236, 0.5974217079, 0.6...","[0.0301832165569067, 0.023524614050984383, 0.0...",0
7,2024-07-22 20:59:50.780,"[0.7798859683, 0.8083992827, 0.1677991605, 0.3...","[0.02800787054002285, 0.024884404614567757, 0....",0
8,2024-07-22 20:59:50.780,"[0.6489823856, 0.4121655702, 0.0318375282, 0.9...","[0.012989213690161705, 0.0409930981695652, 0.0...",0
9,2024-07-22 20:59:50.780,"[0.2535415532, 0.9149270944, 0.7853113608, 0.0...","[0.03925394266843796, 0.02345191314816475, 0.0...",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [12]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,keras-sequential-single-io
created,2024-07-22 20:57:54.646391+00:00
last_updated,2024-07-22 20:59:21.166315+00:00
deployed,False
workspace_id,15
workspace_name,keras-sequential-single-io
arch,x86
accel,none
tags,
versions,"a2523397-0a9f-43c8-b4c2-da4a9493e6f7, 39d9ef3c-778e-4ebf-9aaf-f8c15476cd6c"
steps,keras-sequential-single-io
